# Quick Guide

If you have not yet installed {{ Triumvirate }}, please follow the instructions
in ['Installation'](../installation.rst).

## Using the Python package

### Preamble

Load needed classes and functions from the relevant package modules:

In [1]:
from triumvirate.catalogue import ParticleCatalogue
from triumvirate.logger import setup_logger
from triumvirate.parameters import ParameterSet
from triumvirate.threept import compute_bispec_in_gpp_box

### Optional logger

A logger is useful for program tracking, and can be set up easily and passed
to callables that accept a ``logger`` argument:

In [2]:
trv_logger = setup_logger()

```{seealso}
See the ['Customised Logger'](./Logger.ipynb) tutorial for more details.

### Parameter configuration

The simplest way to initialise an instance of the parameter set class
{py:class}`~triumvirate.parameters.ParameterSet` is by loading an external
``.yml`` file. A template parameter file ``"params_template.yml"`` can be
downloaded from [here](../_static/params_template.yml), but the relevant
parameters need to be changed accordingly for your use.

```{seealso}
See ['Parameter Configuration'](./Configuration.ipynb) and
['Parameter Set'](./Parameters.ipynb) tutorials for more details.
```

In [3]:
import yaml

from triumvirate.parameters import fetch_paramset_template

params_dict = fetch_paramset_template('dict')

for ax_name in ['x', 'y', 'z']:
    params_dict['boxsize'][ax_name] = 1000.
    params_dict['ngrid'][ax_name] = 64

params_dict.update({
    'assignment'    : 'cic',
    'catalogue_type': 'survey',
    'statistic_type': 'bispec',
    'degrees'       : {'ell1': 0, 'ell2': 0, 'ELL': 0},
    'range'         : [0.005, 0.105],
    'num_bins'      : 10,
})

with open("params_template.yml", 'w') as params_file:
    yaml.dump(params_dict, params_file, default_flow_style=False)

Assuming the parameter file is in the current working directory, read the
parameter set from file:

In [4]:
parameter_set = ParameterSet("params_template.yml", logger=trv_logger)

[2023-02-26 02:00:52 (+00:00:00) INFO] Validating parameters... (entering C++)
[2023-02-26 02:00:52 (+00:00:00) INFO] ... validated parameters. (exited C++)
[2023-02-26 02:00:52 (+00:00:00) STAT C++] Parameters validated.


The parameters used here are (possibly printed in a different order):

In [5]:
parameter_set.print()

{'alignment': 'centre',
 'assignment': 'cic',
 'binning': 'lin',
 'boxsize': {'x': 1000.0, 'y': 1000.0, 'z': 1000.0},
 'catalogue_type': 'survey',
 'degrees': {'ELL': 0, 'ell1': 0, 'ell2': 0},
 'directories': {'catalogues': None, 'measurements': None},
 'files': {'data_catalogue': None, 'rand_catalogue': None},
 'form': 'diag',
 'idx_bin': None,
 'interlace': 'false',
 'ngrid': {'x': 64, 'y': 64, 'z': 64},
 'norm_convention': 'particle',
 'num_bins': 10,
 'padfactor': None,
 'padscale': 'box',
 'range': [0.005, 0.105],
 'statistic_type': 'bispec',
 'tags': {'output': None},
 'verbose': 20,
 'wa_orders': {'i': None, 'j': None},
 'npoint': '3pt',
 'space': 'fourier'}


### Catalogue data

We will measure clustering statistics from a single simulation-like catalogue
file ``"mock_catalogue_sim.dat"``, which contains three data columns ``'x'``,
``'y'`` and ``'z'`` for the Cartesian coordinates of particles.

In [6]:
import numpy as np
from nbodykit.cosmology import Cosmology

# Cosmology, matter power spectrum and bias at given redshift
cosmo = Cosmology(
    h=0.6736, Omega0_b=0.04930, Omega0_cdm=0.2645, A_s=2.083e-09, n_s=0.9649
)
redshift = 1.
bias = 2.

# Catalogue properties
density = 5.e-4
boxsize = 1000.

# Catalogue selectors
def cut_to_sphere(coords, boxsize):
    return np.less_equal(np.sqrt(np.sum(coords**2, axis=-1)), boxsize/2.)

# Create simulation-like catalogue, or load if existing.
catalogue_sim_filepath = "mock_catalogue_sim.dat"

try:
    catalogue_sim = np.loadtxt(
        catalogue_sim_filepath,
        dtype=[(axis, np.float64) for axis in ['x', 'y', 'z']]
    )
except FileNotFoundError:
    from nbodykit.lab import LinearPower, LogNormalCatalog
    powspec = LinearPower(cosmo, redshift)
    catalogue_sim = LogNormalCatalog(
        powspec, density, boxsize, bias=bias, Nmesh=256, seed=42
    )
    catalogue_sim['Position'] -= boxsize/2.
    np.savetxt(catalogue_sim_filepath, catalogue_sim['Position'].compute())

Read the catalogue data from file:

In [7]:
# DEMO: warning messages are captured and reprinted here for demonstration.

import warnings  # DEMO

with warnings.catch_warnings(record=True) as demo_warnings:
    catalogue = ParticleCatalogue.read_from_file(
        "mock_catalogue_sim.dat",
        names=['x', 'y', 'z'],
        logger=trv_logger
    )
    for demo_warning in demo_warnings:
        print(demo_warning.message)

[2023-02-26 02:00:58 (+00:00:06) INFO] Original extents of particle coordinates: {'x': (-499.998, 499.998), 'y': (-500.000, 499.998), 'z': (-499.999, 500.000)} (ParticleCatalogue(source=extfile:mock_catalogue_sim.dat)).
[2023-02-26 02:00:58 (+00:00:06) INFO] Catalogue loaded: 499214 particles with total sample weight 499214.000 (ParticleCatalogue(source=extfile:mock_catalogue_sim.dat)).
Catalogue 'nz' field is not provided and thus set to zero, which may raise errors in some computations.
Catalogue 'ws' field is not provided, so is set to unity.
Catalogue 'wc' field is not provided, so is set to unity.


```{hint}
As seen in the warning messages, catalogues should usually contain more
data columns than just the Cartesian coordinates. For a cuboid-box catalogue
of unweighted particles like above, the Cartesian coordinates are the only
necessary input data.
```

```{seealso}
See the ['Particle Catalogue'](./Catalogue.ipynb) tutorial for more details.
```

### Clustering measurements

As an example, we measure the diagonal global plane-parallel bispectrum
monopole $(\ell_1, \ell_2, L) = (0, 0, 0)$ from the simulation-like cuboid-box
catalogue loaded above, and save the result as a ``.txt`` file to the default
output file ``"bk000_diag.txt"``.

Call the function:

In [8]:
results = compute_bispec_in_gpp_box(
    catalogue, paramset=parameter_set,
    save='.txt', logger=trv_logger
)

[2023-02-26 02:00:58 (+00:00:06) INFO] Validating parameters... (entering C++)
[2023-02-26 02:00:58 (+00:00:06) INFO] ... validated parameters. (exited C++)
[2023-02-26 02:00:58 (+00:00:06) STAT C++] Parameters validated.
[2023-02-26 02:00:58 (+00:00:06) INFO] Parameter set have been initialised.
[2023-02-26 02:00:58 (+00:00:06) INFO] Binning has been initialised.
[2023-02-26 02:00:58 (+00:00:06) INFO] Offset extents of particle coordinates: {'x': (0.000, 1000.000), 'y': (0.000, 999.996), 'z': (0.001, 999.997)} (ParticleCatalogue(source=extfile:mock_catalogue_sim.dat)).
[2023-02-26 02:00:58 (+00:00:06) INFO] Catalogue box has been periodised.
[2023-02-26 02:00:58 (+00:00:06) INFO] Inserted missing 'nz' field based on particle count and boxsize.
[2023-02-26 02:00:58 (+00:00:06) INFO] Preparing catalogue for clustering algorithm... (entering C++)
[2023-02-26 02:01:00 (+00:00:12) INFO C++] Catalogue loaded: 499214 particles with total sample weight 499214.000 (source=extdata).
[2023-02-26

The returned `results` is a dictionary containing the coordinate bin centres
and effective/mean coordinates, the number of contributing modes per bin and
(the real and imaginary parts of) the raw measurements including shot noise and
the shot noise component:

In [12]:
# DEMO
from pprint import pprint
pprint(results)

{'bk_raw': array([-9.76634042e+08+5.26214490e-07j,  5.62308286e+09-2.29206433e-07j,
        8.69371377e+08+1.48027213e-07j,  1.28499842e+09+3.94237134e-09j,
        6.68698395e+08-6.55545174e-09j,  6.05966155e+08+3.81950140e-08j,
        3.85704485e+08+7.34849305e-09j,  2.88772358e+08-1.12156484e-08j,
        2.32705251e+08+9.05802470e-09j,  2.00351320e+08+3.02312246e-09j]),
 'bk_shot': array([1.89812841e+08+2.46724756e-09j, 2.06798892e+08+1.59144112e-09j,
       1.50707290e+08+3.02190775e-10j, 1.24950405e+08-3.72568273e-10j,
       1.06980049e+08-7.86374911e-11j, 9.44867868e+07+3.44890456e-11j,
       8.04644721e+07+3.79496799e-11j, 6.84708228e+07-1.63157897e-10j,
       5.94748920e+07-1.02103091e-10j, 5.12239555e+07+1.27372284e-10j]),
 'k1bin': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ]),
 'k1eff': array([0.01149964, 0.02047114, 0.03052422, 0.04062536, 0.05033379,
       0.06040971, 0.07034279, 0.08025683, 0.09014322, 0.10008191]),
 'k2bin': array([0.01, 0.02,

The saved file contains a header with summarY information about the parameters
used and the input data as well as intermediary quantities:

In [10]:
# DEMO
with open("bk000_diag.txt", 'r') as results_file:
    print(results_file.read())

# Catalogue source: extfile:mock_catalogue_sim.dat
# Catalogue size: 499214 particles of total sample weight 499214.000
# Catalogue particle extents: ([0.000, 1000.000], [0.000, 999.996], [0.001, 999.997])
# Box size: (1000.000, 1000.000, 1000.000)
# Box alignment: centre
# Mesh number: (64, 64, 64)
# Mesh assignment and interlacing: cic, false
# Normalisation factor: 8.037846928e+00 (particle-based, used), 2.208590694e+00 (mesh-based, alternative)
# [0] k1_cen, [1] k1_eff, [2] k2_cen, [3] k2_eff, [4] nmodes, [5] Re{bk000_raw}, [6] Im{bk000_raw}, [7] Re{bk000_shot}, [8] Im{bk000_shot}
1.000000000e-02	1.149964290e-02	1.000000000e-02	1.149964290e-02	        56	-9.766340417e+08	 5.262144896e-07	 1.898128415e+08	 2.467247558e-09
2.000000000e-02	2.047114034e-02	2.000000000e-02	2.047114034e-02	       194	 5.623082855e+09	-2.292064332e-07	 2.067988919e+08	 1.591441123e-09
3.000000000e-02	3.052421724e-02	3.000000000e-02	3.052421724e-02	       488	 8.693713775e+08	 1.480272132e-07	 1.507072903e

In [11]:
!rm bk000_diag.txt

```{seealso}
See the ['Clustering Measurements'](./Measurements.ipynb) tutorial for more details.
```

## Running the C++ program